In [ ]:
# !git clone https://github.com/KCDichDaNgu/KC4.0_MultilingualNMT.git
from google.colab import drive
drive.mount('/content/drive')

# lấy project từ drive
!cp -r /content/drive/MyDrive/KC4.0_MultilingualNMT /content/

#xoá thư mục vừa tạo trên colab:
# !rm -rf /content/KC4.0_MultilingualNMT


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ghi đè vào project trên drive
!cp -ru /content/KC4.0_MultilingualNMT/* /content/drive/MyDrive/KC4.0_MultilingualNMT/

thêm dữ liệu VLSP2023 dịch Việt - Lào

In [ ]:
# !mkdir -p /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo

# !cp /content/drive/MyDrive/VLSP2023_vi_lo/train2023.vi /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.vi
# !cp /content/drive/MyDrive/VLSP2023_vi_lo/train2023.lo /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.lo

# # đổi tên file dev thành valid
# !cp /content/drive/MyDrive/VLSP2023_vi_lo/dev2023.vi /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid.vi
# !cp /content/drive/MyDrive/VLSP2023_vi_lo/dev2023.lo /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid.lo


**Requirements**

In [ ]:
!pip install torch

In [ ]:
!pip install torchtext==0.6.0

In [ ]:
!pip install nltk pyvi spacy dill

In [ ]:
!pip install PyYAML==6.0.1

**Chọn tokenizer**

In [ ]:
import sentencepiece as spm
!pip list | grep sentencepiece

sentencepiece                         0.2.0


**Train Tokenize**

In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
    input="/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.vi,/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.lo",
    model_prefix="vi_lo_bpe",
    vocab_size=8000,
    character_coverage=1.0,
    model_type="bpe"
)

nhét vi_lo_bpe.model và .vocab vào repo cho tiện

In [ ]:
# # !find /content/drive/MyDrive -name "vi_lo_bpe.*"
# !ls -lh /content/vi_lo_bpe.*


-rw-r--r-- 1 root root 370K May 14 11:27 /content/vi_lo_bpe.model
-rw-r--r-- 1 root root 121K May 14 11:27 /content/vi_lo_bpe.vocab


In [ ]:
# !mv /content/vi_lo_bpe.* /content/KC4.0_MultilingualNMT/

# **Thực hiện tokenize**

In [ ]:
# import sentencepiece as spm

# sp = spm.SentencePieceProcessor()
# sp.load("/content/KC4.0_MultilingualNMT/vi_lo_bpe.model")

True

In [ ]:
# def tokenize_file(input_path, output_path):
#     with open(input_path, 'r', encoding='utf-8') as fin, open(output_path, 'w', encoding='utf-8') as fout:
#         for line in fin:
#             tokens = sp.encode(line.strip(), out_type=str)
#             fout.write(' '.join(tokens) + '\n')

In [ ]:
# # Train
# tokenize_file("/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.vi",  "/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train_tok.vi")
# tokenize_file("/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.lo",  "/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train_tok.lo")

# # Validation
# tokenize_file("/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid.vi", "/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid_tok.vi")
# tokenize_file("/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid.lo", "/content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid_tok.lo")

Ghi đè lên dữ liệu cũ để bắt đầu training

In [ ]:
# !mv /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train_tok.vi /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.vi
# !mv /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train_tok.lo /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.lo

# !mv /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid_tok.vi /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid.vi
# !mv /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid_tok.lo /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/valid.lo

# **Training**

Đang để YaML bản cao hơn requirements nên có cái này

In [ ]:
import yaml

# Monkey patch: thay thế yaml.load bằng phiên bản an toàn có Loader
def safe_yaml_load_patch():
    orig_load = yaml.load
    def safe_load_with_loader(*args, **kwargs):
        if 'Loader' not in kwargs:
            kwargs['Loader'] = yaml.SafeLoader
        return orig_load(*args, **kwargs)
    yaml.load = safe_load_with_loader

safe_yaml_load_patch()


In [ ]:
!ls -lh /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/
!head -n 5 /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.vi
!head -n 5 /content/KC4.0_MultilingualNMT/data/vlsp_vi_lo/train.lo

total 49M
-rw------- 1 root root  28M May 14 15:38 train.lo
-rw------- 1 root root  21M May 14 15:38 train.vi
-rw------- 1 root root 511K May 14 15:38 valid.lo
-rw------- 1 root root 381K May 14 15:38 valid.vi
▁Nếu ▁anh ▁chị ▁cảm ▁thấy ▁e ▁ngại ▁khi ▁giới ▁thiệu ▁về ▁tin ▁mừng , ▁hãy ▁tập ▁bắt ▁chuyện ▁mà ▁không ▁nhất ▁thiết ▁làm ▁chứng .
▁H ẳ n ▁nhiều ▁người ▁thấp ▁kém ▁e ▁sợ ▁và ▁tránh ▁mặt ▁những ▁nhà ▁lãnh ▁đạo ▁ấy , ▁thay ▁vì ▁xin ▁họ ▁giúp ▁đỡ ▁hoặc ▁chỉ ▁dẫn .
▁21 ▁B ấy ▁giờ , ▁khi ▁ho àng ▁hậu ▁trông ▁thấy ▁vẻ ▁kinh ▁hãi ▁của ▁các ▁tôi ▁t ớ , ▁bà ▁cũng ▁cảm ▁thấy ▁sợ ▁hãi ▁vô ▁cùng , ▁e ▁rằng ▁điều ▁dữ ▁sẽ ▁đến ▁với ▁bà .
▁Q uyết ▁định ▁để ▁thay ▁đổi ▁thuộc ▁vào ▁chính ▁các ▁anh ▁chị ▁em , ▁và ▁chỉ ▁một ▁mình ▁các ▁anh ▁chị ▁em ▁mà ▁thôi .
▁Những ▁lựa ▁chọn ▁của ▁các ▁em ▁ngay ▁trước ▁mắt ▁là ▁quan ▁trọng ▁vĩnh ▁viễn .
▁ຖ້າ ▁ເຈົ້າ ▁ຮູ້ສຶກ ▁ຢ້ານ ▁ທີ່ ▁ຈະ ▁ປະກາດ ▁ຂໍ ▁ໃຫ້ ▁ຊ ້ອມ ▁ເວົ້າ ▁ໃນ ▁ແບບ ▁ທີ່ ▁ຍັງ ▁ບໍ່ ▁ມີ ▁ເປົ້າ ▁ຫມາຍ ▁ປະກາດ .
▁ຄົນ ▁ຕ ໍ່າ ▁ຕ ້ອຍ ▁ຈໍານວນ ▁ຫຼາຍ ▁ຄົງ ▁ຈະ ▁ຫຼ 

In [ ]:
%cd /content/KC4.0_MultilingualNMT

!mkdir -p /content/drive/MyDrive/KC4_checkpoint/vi_lo.model

!python -m bin.main train \
  --model Transformer \
  --config config/bilingual_prototype.yml \
  --model_dir /content/drive/MyDrive/KC4_checkpoint/vi_lo.model


/content/KC4.0_MultilingualNMT
<frozen runpy>:128: RuntimeWarning: 'bin.main' found in sys.modules after import of package 'bin', but prior to execution of 'bin.main'; this may result in unpredictable behaviour
Config specified, copying all to model dir
Loaded path is a list of locations. Load in the order received, overriding and merging as needed.
Building vocab from received data.
No checkpoint found, start from beginning.
[Wed, 14 May 2025 15:41:47 INFO] .en * src vocab size = 14591
[Wed, 14 May 2025 15:41:47 INFO] .vi * tgt vocab size = 6839
[Wed, 14 May 2025 15:41:47 INFO] Building model...
Zero checkpoint detected, reinitialize the model
[Wed, 14 May 2025 15:41:52 INFO] Transformer(
  (encoder): Encoder(
    (embed): Embedding(14591, 512)
    (pe): PositionalEncoder(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (norm_1): Norm()
        (norm_2): Norm()
        (attn): MultiHeadAttention(
          (q_linear

In [ ]:
!cp models/vi_lo.model/checkpoint_last.pt /content/drive/MyDrive/

cp: cannot stat 'models/vi_lo.model/checkpoint_last.pt': No such file or directory


Nếu sao lưu config và tokenizer